# Overview

## What to Predict

- Stage 1 - You should submit predicted probabilities for every possible matchup in the past 5 NCAA® tournaments (seasons 2015-2019).
- Stage 2 - You should submit predicted probabilities for every possible matchup before the 2020 tournament begins.

Refer to the [Timeline page](https://www.kaggle.com/c/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/overview/timeline) for specific dates. In both stages, the sample submission will tell you which games to predict.

# Import Packages

In [5]:
import os

import pandas as pd
import numpy as np

import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 20, 6


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Data Download

In [6]:
# Downloading the Data using Kaggle API
!kaggle competitions download -c google-cloud-ncaa-march-madness-2020-division-1-mens-tournament

google-cloud-ncaa-march-madness-2020-division-1-mens-tournament.zip: Skipping, found more recently modified local copy (use --force to force download)


# Data Import

In [7]:
def get_file_list(datapath):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(datapath)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(datapath, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + get_file_list(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [9]:
main_path = 'F:\\OneDrive - Georgia State University\\Data Science\\Competition\\Data\\Google Cloud & NCAA® ML Competition 2020-NCAAM\\'
sub_folders = os.listdir(main_path)
datapath = main_path + sub_folders[0] + '\\'

# Get the list of all files in directory tree at given path
data_list = get_file_list(datapath)

data_list

['F:\\OneDrive - Georgia State University\\Data Science\\Competition\\Data\\Google Cloud & NCAA® ML Competition 2020-NCAAM\\Data Section 1 - The Basics\\MNCAATourneyCompactResults.csv',
 'F:\\OneDrive - Georgia State University\\Data Science\\Competition\\Data\\Google Cloud & NCAA® ML Competition 2020-NCAAM\\Data Section 1 - The Basics\\MNCAATourneySeeds.csv',
 'F:\\OneDrive - Georgia State University\\Data Science\\Competition\\Data\\Google Cloud & NCAA® ML Competition 2020-NCAAM\\Data Section 1 - The Basics\\MRegularSeasonCompactResults.csv',
 'F:\\OneDrive - Georgia State University\\Data Science\\Competition\\Data\\Google Cloud & NCAA® ML Competition 2020-NCAAM\\Data Section 1 - The Basics\\MSampleSubmissionStage1_2020.csv',
 'F:\\OneDrive - Georgia State University\\Data Science\\Competition\\Data\\Google Cloud & NCAA® ML Competition 2020-NCAAM\\Data Section 1 - The Basics\\MSeasons.csv',
 'F:\\OneDrive - Georgia State University\\Data Science\\Competition\\Data\\Google Cloud & NC

In [103]:
tourney_compact_result_M = pd.read_csv(datapath + 'MNCAATourneyCompactResults.csv')
tourney_seed_M = pd.read_csv(datapath + 'MNCAATourneySeeds.csv')
regular_compact_result_M = pd.read_csv(datapath + 'MRegularSeasonCompactResults.csv')
season_M = pd.read_csv(datapath + 'MSeasons.csv')
teams_M = pd.read_csv(datapath + 'MTeams.csv')

submission_file = pd.read_csv(datapath + 'MSampleSubmissionStage1_2020.csv')

# Data Exploratory Analysis

    - for each data we will:
        - handle missing values
        - find-out meaningful information that can potentially affect the result of the game
        - create a new feature within the data

## Check missing data and data types

In [107]:
# create a column indicating that the data is from regular seasons or tournaments.
tourney_compact_result_M['regular_or_tourney'] = "T"
regular_compact_result_M['regular_or_tourney'] = "R"

# Concatenate the tourney and regular compact results.
entire_compact_result_M = pd.concat([regular_compact_result_M, tourney_compact_result_M])

# Create a match up column with lower TeamID comes first.
entire_compact_result_M['match_up'] = entire_compact_result_M[['WTeamID', 'LTeamID']].values.tolist()
entire_compact_result_M['match_up'] = entire_compact_result_M['match_up'].apply(sorted)

# check the non-null count and the data types. 
entire_compact_result_M.info()
entire_compact_result_M

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163803 entries, 0 to 2250
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Season              163803 non-null  int64 
 1   DayNum              163803 non-null  int64 
 2   WTeamID             163803 non-null  int64 
 3   WScore              163803 non-null  int64 
 4   LTeamID             163803 non-null  int64 
 5   LScore              163803 non-null  int64 
 6   WLoc                163803 non-null  object
 7   NumOT               163803 non-null  int64 
 8   regular_or_tourney  163803 non-null  object
 9   match_up            163803 non-null  object
dtypes: int64(7), object(3)
memory usage: 13.7+ MB


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,regular_or_tourney,match_up
0,1985,20,1228,81,1328,64,N,0,R,"[1228, 1328]"
1,1985,25,1106,77,1354,70,H,0,R,"[1106, 1354]"
2,1985,25,1112,63,1223,56,H,0,R,"[1112, 1223]"
3,1985,25,1165,70,1432,54,H,0,R,"[1165, 1432]"
4,1985,25,1192,86,1447,74,H,0,R,"[1192, 1447]"
...,...,...,...,...,...,...,...,...,...,...
2246,2019,146,1120,77,1246,71,N,1,T,"[1120, 1246]"
2247,2019,146,1277,68,1181,67,N,0,T,"[1181, 1277]"
2248,2019,152,1403,61,1277,51,N,0,T,"[1277, 1403]"
2249,2019,152,1438,63,1120,62,N,0,T,"[1120, 1438]"


In [104]:
entire_compact_result_M['match_up'] = entire_compact_result_M[['WTeamID', 'LTeamID']].values.tolist()
entire_compact_result_M['match_up'].apply(sorted)

entire_compact_result_M

0       [1228, 1328]
1       [1106, 1354]
2       [1112, 1223]
3       [1165, 1432]
4       [1192, 1447]
            ...     
2246    [1120, 1246]
2247    [1181, 1277]
2248    [1277, 1403]
2249    [1120, 1438]
2250    [1403, 1438]
Name: match_up, Length: 163803, dtype: object

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,regular_or_tourney,match_up,match_ups
0,1985,20,1228,81,1328,64,N,0,R,"[1228, 1328]","[1228, 1328]"
1,1985,25,1106,77,1354,70,H,0,R,"[1106, 1354]","[1106, 1354]"
2,1985,25,1112,63,1223,56,H,0,R,"[1112, 1223]","[1112, 1223]"
3,1985,25,1165,70,1432,54,H,0,R,"[1165, 1432]","[1165, 1432]"
4,1985,25,1192,86,1447,74,H,0,R,"[1192, 1447]","[1192, 1447]"
...,...,...,...,...,...,...,...,...,...,...,...
2246,2019,146,1120,77,1246,71,N,1,T,"[1120, 1246]","[1120, 1246]"
2247,2019,146,1277,68,1181,67,N,0,T,"[1277, 1181]","[1181, 1277]"
2248,2019,152,1403,61,1277,51,N,0,T,"[1403, 1277]","[1277, 1403]"
2249,2019,152,1438,63,1120,62,N,0,T,"[1438, 1120]","[1120, 1438]"


In [85]:
# Just to make sure, check if there is any team never won or lost.

a = teams_M['TeamID']
b = entire_compact_result_M['WTeamID']
c = entire_compact_result_M['LTeamID']

print(f'never won: {np.setdiff1d(a,b)}')
print(f'never lost: {np.setdiff1d(a,c)}')

teams_M[teams_M['TeamID']==1467]

never won: [1467]
never lost: [1467]


,TeamID,TeamName,FirstD1Season,LastD1Season
366,1467,Merrimack,2020,2020


In [62]:
teams_M.info()
teams_M

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TeamID         367 non-null    int64 
 1   TeamName       367 non-null    object
 2   FirstD1Season  367 non-null    int64 
 3   LastD1Season   367 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 11.6+ KB


,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2020
1,1102,Air Force,1985,2020
2,1103,Akron,1985,2020
3,1104,Alabama,1985,2020
4,1105,Alabama A&M,2000,2020
...,...,...,...,...
362,1463,Yale,1985,2020
363,1464,Youngstown St,1985,2020
364,1465,Cal Baptist,2019,2020
365,1466,North Alabama,2019,2020


In [61]:
entire_compact_result_M['count'] = 1
winning_data = entire_compact_result_M.groupby(['WTeamID','Season']).sum()
losing_data = entire_compact_result_M.groupby(['LTeamID','Season']).sum()

intra_season_winning_data = entire_compact_result_M.groupby(['WTeamID', 'DayNum']).sum()
intra_season_losing_data = entire_compact_result_M.groupby(['LTeamID', 'DayNum']).sum()


seasonal_data = entire_compact_result_M.groupby(['Season','DayNum','WTeamID']).sum()


winning_data['count'].max()
losing_data['count'].max()

intra_season_winning_data
intra_season_losing_data
seasonal_data



38

31

Season  WScore  LTeamID  LScore  NumOT  count
WTeamID DayNum                                               
1101    4         2019      94     1117      73      0      1
        10        2019      67     1176      61      0      1
        17        4036     137     2495     113      0      2
        18        2019      73     1334      71      0      1
        19        2019      60     1415      48      0      1
...                ...     ...      ...     ...    ...    ...
1466    72        2019      91     1239      88      0      1
        75        2019      63     1391      62      0      1
        80        2019      76     1244      71      0      1
        107       2019      76     1244      61      0      1
        113       2019      69     1239      55      0      1

[34474 rows x 6 columns]

Season  WTeamID  WScore  LScore  NumOT  count
LTeamID DayNum                                               
1101    5         2014     1182      94      75      0      1
        7         2014     1382      75      47      0      1
        9         2014     1268      67      44      0      1
        13        2014     1234     103      41      0      1
        14        2018     1117      83      69      0      1
...                ...      ...     ...     ...    ...    ...
1466    96        2019     1316      82      73      0      1
        103       2019     1251      80      70      0      1
        110       2019     1391      63      60      0      1
        116       2019     1252      87      75      0      1
        119       2019     1316      76      66      0      1

[36068 rows x 6 columns]

WScore  LTeamID  LScore  NumOT  count
Season DayNum WTeamID                                       
1985   20     1228         81     1328      64      0      1
       25     1106         77     1354      70      0      1
              1112         63     1223      56      0      1
              1165         70     1432      54      0      1
              1192         86     1447      74      0      1
...                       ...      ...     ...    ...    ...
2019   146    1120         77     1246      71      1      1
              1277         68     1181      67      0      1
       152    1403         61     1277      51      0      1
              1438         63     1120      62      0      1
       154    1438         85     1403      77      1      1

[163803 rows x 5 columns]

In [32]:
tourney_seed_M.info()
tourney_seed_M

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2286 entries, 0 to 2285
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Season  2286 non-null   int64 
 1   Seed    2286 non-null   object
 2   TeamID  2286 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 53.7+ KB


,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
2281,2019,Z12,1332
2282,2019,Z13,1414
2283,2019,Z14,1330
2284,2019,Z15,1159


In [18]:
season_M.info()
season_M

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Season   36 non-null     int64 
 1   DayZero  36 non-null     object
 2   RegionW  36 non-null     object
 3   RegionX  36 non-null     object
 4   RegionY  36 non-null     object
 5   RegionZ  36 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.8+ KB


,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/2/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast
5,1990,10/30/1989,East,Midwest,Southeast,West
6,1991,10/29/1990,East,Southeast,Midwest,West
7,1992,11/4/1991,East,West,Midwest,Southeast
8,1993,11/2/1992,East,Midwest,Southeast,West
9,1994,11/1/1993,East,Southeast,Midwest,West


## Concatenate Regular Season Results and Tourney Results

- per team
    - score trend throughout a season, throughout years. 
    - score trend depend on cities.

# Feature Engineering

- desired output:
    - index: each team id
    - Season:
    - DayNum:
    - CityID:
    - opponent:
    - Score:
    - Opp_Score:
    - result: Result (W or L)

# Reference
- Primary: [google-cloud-ncaa-march-madness-2020-division-1-mens-tournament](https://www.kaggle.com/c/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament)
- Secondary:

## Section 0 - Submission File

In [6]:
submission_file

,ID,Pred
0,2015_1107_1112,0.5
1,2015_1107_1116,0.5
2,2015_1107_1124,0.5
3,2015_1107_1125,0.5
4,2015_1107_1129,0.5
...,...,...
11385,2019_1449_1459,0.5
11386,2019_1449_1463,0.5
11387,2019_1458_1459,0.5
11388,2019_1458_1463,0.5


In [70]:
# Spliting ID into 3 different columns
submission_file['Season'] = submission_file['ID'].apply(lambda x: x.split("_")[0])
submission_file['lower_team'] = submission_file['ID'].apply(lambda x: x.split("_")[1])
submission_file['higher_team'] = submission_file['ID'].apply(lambda x: x.split("_")[2])

# Reorder the columns to move the target variable to the end
cols = ["ID", "Season", "lower_team", "higher_team", "Pred"]

# apply the corrected column order to the data frame
submission_file = submission_file[cols]
submission_file

,ID,Season,lower_team,higher_team,Pred
0,2015_1107_1112,2015,1107,1112,0.5
1,2015_1107_1116,2015,1107,1116,0.5
2,2015_1107_1124,2015,1107,1124,0.5
3,2015_1107_1125,2015,1107,1125,0.5
4,2015_1107_1129,2015,1107,1129,0.5
...,...,...,...,...,...
11385,2019_1449_1459,2019,1449,1459,0.5
11386,2019_1449_1463,2019,1449,1463,0.5
11387,2019_1458_1459,2019,1458,1459,0.5
11388,2019_1458_1463,2019,1458,1463,0.5
